In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import gensim
# Importing necessary libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import string
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors
import json
import sys
import os
import os.path
os.environ['TZ'] = 'Asia/Calcutta'
from google.cloud import bigquery
import multiprocessing
from time import time
import warnings
warnings.filterwarnings("ignore")
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/foram/Desktop/Foram/MyGlamm - Data Storage-3800e204ff33.json"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = base_dir + "/core/.mg_in_creds.json"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/centos/data_transform/core/.mg_in_creds.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Mouna Reddy/OneDrive/Documents/mouna_reddy.json"
client=bigquery.Client()

In [96]:
query1 = ('''
            with raw_data as (select DISTINCT product_id , sku,brand_name,parent_category_name,category_name, product_name,
shadelabel,coverage,concern,formulation,finish,skintype,ingredient,color, base_price,vendor_code
from  `myglamm-india.master_tables.master_product_table_v2` 
where type != 'combo' and hidden = false
and product_status = 'Active' and brand_status = 'Active' 
and parent_category_status = 'Active' and
vendor_code = 'mgp'),

vcode as (SELECT oa.* , ct.* FROM raw_data oa
left join 
(SELECT sourceid,data.metadata.keywords,data.metadata.title,data.metadata.description,
data.content.subtitle  FROM `myglamm-india.charts_in_v2.product_content_details`) as ct 
on oa.product_id = ct.sourceid), 

orders as (SELECT  distinct product_name, count(distinct order_id) as total_orders 
FROM `myglamm-india.charts_in_v2.product_table`
where vendor_code = 'mgp'
and product_name != 'ShippingCharges'
and lower(channel) = lower('ONLINE')
and order_type = 'Positive' 
and product_status = 'Active'
and lower(order_category) != lower('Child')
and status_id not in (11,17)
and lower(product_name) not like lower('%offer price%')
group by product_name 
order by total_orders desc),

prod_orders as (SELECT vc.* , od.total_orders FROM vcode vc
left join 
(SELECT product_name,total_orders  FROM orders) as od 
on vc.product_name = od.product_name)

select DISTINCT sku,brand_name,parent_category_name,category_name, product_name,total_orders,
shadelabel,coverage,concern,formulation,finish,skintype,ingredient,color, base_price,vendor_code,
sourceid, keywords, title, description, subtitle from prod_orders;

          ''')
df = client.query(query1).to_dataframe()

In [97]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})

In [98]:
missing_value_df = missing_value_df.sort_values('percent_missing', ascending=False)
missing_value_df

,column_name,percent_missing
concern,concern,84.488108
ingredient,ingredient,71.458118
color,color,56.670114
coverage,coverage,48.810755
skintype,skintype,47.259566
finish,finish,45.087901
formulation,formulation,43.019648
shadelabel,shadelabel,15.201655
total_orders,total_orders,2.792141
vendor_code,vendor_code,0.000000


In [99]:
def join_desc(x): return str(x['product_name'])+' '+str(x['parent_category_name'])+' '\
                          +str(x['title'])+' '+str(x['subtitle'])+' '\
# +str(x['title'])+' '+str(x['description'])+' '\
#                      +str(x['shadelabel'])+' '+str(x['finish'])+' '+str(x['formulation'])+' '\
#                      +str(x['coverage'])+' '+str(x['skintype'])+' '\
                    #+str(df1['color'])+' '+str(df1['concern'])+' '+str(df1['ingredient'])+' '
df['details'] = df.apply(join_desc, axis=1)

In [100]:
# df

In [101]:
df.product_name[0]

'MyGlamm Manicure Kit'

In [102]:
df.parent_category_name[0]

'Makeup Brushes And Accessories'

In [103]:
df.title[0]

'Manicure Tools Kit -  5 Piece Nail Manicure Kit | MyGlamm'

In [104]:
df.subtitle[0]

'Special Edition Manicure Set'

In [105]:
df.details[0]

'MyGlamm Manicure Kit Makeup Brushes And Accessories Manicure Tools Kit -  5 Piece Nail Manicure Kit | MyGlamm Special Edition Manicure Set '

In [106]:
# df.keywords[0]

In [107]:
# df.description[0]

In [108]:
df1 = df.drop_duplicates(subset=['sku'], keep='last').reset_index(drop=True)

In [109]:
# df1["lower_details"] = df1["details"].apply(lambda x : x.lower())

In [110]:
# df1["lower_details"]

In [111]:
#Utitlity functions for removing ASCII characters, converting lower case, removing stop words, html and punctuation from description

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df1['cleaned'] = df1['details'].apply(_removeNonAscii)
df1['cleaned'] = df1.cleaned.apply(func = make_lower_case)
df1['cleaned'] = df1.cleaned.apply(func = remove_stop_words)
df1['cleaned'] = df1.cleaned.apply(func=remove_punctuation)
df1['cleaned'] = df1.cleaned.apply(func=remove_html)

In [112]:
df1['cleaned'][0]

'myglamm manicure kit makeup brushes accessories manicure tools kit 5 piece nail manicure kit myglamm special edition manicure set'

In [113]:
# size: The number of dimensions of the embeddings and the default is 100.
# window: The maximum distance between a target word and words around the target word. The default window is 5.
# min_count: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 5.
# workers: The number of partitions during training and the default workers is 3.
# sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.

In [114]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    Takes a list of numpy arrays containing documents.
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    Args:
        sentences: iterator for sentences
    
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences,min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     sg = 1,
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

In [115]:
w2vec = get_word2vec(MySentences(df1['cleaned'].values),'w2vmodel_mgp')

Found w2vmodel_mgp


In [116]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.vectors[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [117]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
mean_embedded = mean_embedding_vectorizer.fit_transform(df1['cleaned'])

In [118]:
mean_embedded.shape

(802, 300)

In [119]:
# save the vectors on g cloud

In [120]:
# finding cosine similarity for the vectors

cosine_similarities = cosine_similarity(mean_embedded, mean_embedded)
cosine_similarities

array([[0.9999999 , 0.29986548, 0.2777629 , ..., 0.39454362, 0.39391133,
        0.4178436 ],
       [0.29986548, 0.9999998 , 0.38673788, ..., 0.38603792, 0.40122914,
        0.38256517],
       [0.2777629 , 0.38673788, 1.        , ..., 0.41539142, 0.4257528 ,
        0.38935667],
       ...,
       [0.39454362, 0.38603792, 0.41539142, ..., 0.99999994, 0.986611  ,
        0.96505594],
       [0.39391133, 0.40122914, 0.4257528 , ..., 0.986611  , 1.        ,
        0.9641842 ],
       [0.4178436 , 0.38256517, 0.38935667, ..., 0.96505594, 0.9641842 ,
        0.99999964]], dtype=float32)

In [121]:
cosine_similarities.shape

(802, 802)

In [122]:
# base price
# appling a booster 

In [123]:
df1.set_index('sku', inplace = True)

In [124]:
# df1.set_index('sku', inplace = True)
indices = pd.Series(df1.index)

In [125]:
#https://www.statology.org/normalize-data-between-0-and-1/
def NormalizeData(min_scale_num,max_scale_num,var):
    return ((var - min(var)) / (max(var) - min(var)))

# You can input your range
df1['base_price_norm'] = NormalizeData(0,1,df1.base_price.astype(float)) # scaled between 0,1

In [126]:
def remove_same_category_products(x,category_name):
    recomm_skus=df1[df1.index.isin(x)]
    recomm_skus=recomm_skus.drop_duplicates(subset=['category_name'], keep='first').index.values
    return recomm_skus

In [127]:
def recommendations(name, remove_filter,cosine_similarities,variant1,variant2):
    global recommended_prods
    recommended_prods = []
    recommended_indices = []
    recommeded_scores = []
    recommeded_order_sku = []
    # gettin the index of the product that matches the name
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    # getting the indexes of all products 
    all_indexes = list(score_series.index)
    
    for i in all_indexes:
        if df1['product_name'][idx] != df1['product_name'][i]:
            if df1[remove_filter][idx] == df1[remove_filter][i]:
                recommended_prods.append((df1.index)[i])
                recommended_indices.append(i)
                recommeded_scores.append(score_series[i])
                
    df = df1[df1.index.isin(recommended_prods)]
    df = df.reindex(recommended_prods)
    df['score_cs'] = recommeded_scores
    df=df[df['score_cs']>0.10]
    
    
# variant 1 using best sellers

    df['order_dist'] = df['total_orders']/ df['total_orders'].sum()
    df['order_score'] = df['order_dist'] * df['score_cs']
    

# variant 2 using base price
    df['bp_log'] = 1/np.log10(df['base_price'])
    df['base_price_score'] = df['bp_log'] * df['score_cs']
    
    
    df = df.sort_values(by=[variant1],ascending=False)
    recommeded_order_sku_v1 = df.index
    score_cs = df.score_cs
    order_score = df.order_score
    
    #df = df.sort_values(by=[variant2],ascending=False)

    #recommeded_order_sku_v2 = [df.index, df.score_cs, df.order_score]
    
    return recommeded_order_sku_v1,score_cs,order_score 

In [128]:
df1['recommended_sku_v1'] = np.empty((len(df1), 0)).tolist()
df1['score_cs'] = np.empty((len(df1), 0)).tolist()
df1['order_score'] = np.empty((len(df1), 0)).tolist()

# df1['recommended_sku_v2'] = np.empty((len(df1), 0)).tolist()

In [129]:
df1

,brand_name,parent_category_name,category_name,product_name,total_orders,shadelabel,coverage,concern,formulation,finish,...,keywords,title,description,subtitle,details,cleaned,base_price_norm,recommended_sku_v1,score_cs,order_score
sku,,,,,,,,,,,,,,,,,,,,,
STPL1701001,MyGlamm,Makeup Brushes And Accessories,MyGlamm Manicure Kit,MyGlamm Manicure Kit,80.0,None,None,None,None,None,...,"manicure kit products, manicure kit list, how ...",Manicure Tools Kit - 5 Piece Nail Manicure Ki...,This super cute Myglamm's manicure and pedicur...,Special Edition Manicure Set,MyGlamm Manicure Kit Makeup Brushes And Access...,myglamm manicure kit makeup brushes accessorie...,0.001625,[],[],[]
8906095884223,Manish Malhotra,Lip Makeup | Eye Makeup | Face Makeup,Manish Malhotra Metallic Chalk Duo,Manish Malhotra Metallic Chalk Duo - Inferno,746.0,Inferno,Full,None,Powder,Metallic-matte,...,"Manish Malhotra Metallic Chalk Duo, Manish Mal...",Buy Manish Malhotra Metallic Chalk Duo - Infer...,Buy MyGlamm Manish Malhotra Metallic Chalk Duo...,"Multifunctional Metallic Lip, Eye & Cheek Chalk",Manish Malhotra Metallic Chalk Duo - Inferno L...,manish malhotra metallic chalk duo inferno lip...,0.006626,[],[],[]
8906095886050,Treat Love Cared,Face Makeup,Treat Love Care Oil Control Foundation,Treat Love Care Oil Control Foundation - Renew,1987.0,Renew,None,None,None,None,...,"renew, foundation, renew oil control foundatio...",Buy Treat Love Care Oil Control Foundation - R...,Buy MyGlamm Treat Love Care Oil Control Founda...,"Liquid Foundation With Buildable Coverage, Enr...",Treat Love Care Oil Control Foundation - Renew...,treat love care oil control foundation renew f...,0.004851,[],[],[]
8906095886036,Treat Love Cared,Face Makeup,Treat Love Care 24 HR Anti-Pollution Foundation,Treat Love Care 24 HR Anti-Pollution Foundatio...,1025.0,Pure,None,None,None,None,...,"anti pollution foundation, myglamm anti-pollut...",Buy Treat Love Care 24 HR Anti-pollution Found...,Buy MyGlamm Treat Love Care 24 HR Anti-polluti...,"Liquid Foundation With Buildable Coverage, Enr...",Treat Love Care 24 HR Anti-Pollution Foundatio...,treat love care 24 hr anti pollution foundatio...,0.004851,[],[],[]
8906095887699,WIPEOUT,Body | Body,MyGlamm Wipeout Germ Killing Face Wash - Rose ...,MyGlamm Wipeout Germ Killing Face Wash - Rose ...,20170.0,None,None,None,None,None,...,,Buy Wipeout Germ Killing Rose & Honey Face Was...,Buy Wipeout germ killing rose & honey face was...,Rose & honey Infused Germ-killing & Purifying ...,MyGlamm Wipeout Germ Killing Face Wash - Rose ...,myglamm wipeout germ killing face wash rose ho...,0.000470,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8906095889440,Manish Malhotra,Nail Polish Sets,Manish Malhotra Gel Finish Nail Lacquer - The ...,Manish Malhotra Gel Finish Nail Lacquer - The ...,233.0,Red Mystique,None,None,None,None,...,,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,Buy Manish Malhotra gel finish nail lacquer - ...,"Ultra-Glossy, Long-Lasting Gel Nail Enamel",Manish Malhotra Gel Finish Nail Lacquer - The ...,manish malhotra gel finish nail lacquer love e...,0.001350,[],[],[]
8906095889464,Manish Malhotra,Nail Polish Sets,Manish Malhotra Gel Finish Nail Lacquer - The ...,Manish Malhotra Gel Finish Nail Lacquer - The ...,967.0,Sienna Crush,None,None,None,None,...,,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,Buy Manish Malhotra gel finish nail lacquer - ...,"Ultra-Glossy, Long-Lasting Gel Nail Enamel",Manish Malhotra Gel Finish Nail Lacquer - The ...,manish malhotra gel finish nail lacquer love e...,0.001350,[],[],[]
8906095889501,Manish Malhotra,Nail Polish Sets,Manish Malhotra Gel Finish Nail Lacquer - The ...,Manish Malhotra Gel Finish Nail Lacquer - The ...,759.0,Persian Rosé,None,None,None,None,...,,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,"Ultra-Glossy, Long-Lasting Gel Nail Enamel",Manish Malhotra Gel Finish Nail Lacquer -

In [130]:
# df1.index

In [131]:
# recommeded_order_sku_v1 , recommeded_order_sku_v2 = recommendations(indices[i],'parent_category_name',cosine_similarities,variant1,variant2)

In [132]:
# recommeded_order_sku_v2

In [133]:
i=100

In [134]:
category_name=df1[df1.index==indices[i]]['category_name'][indices[i]]
recommeded_order_sku_v1,score_cs,order_score = recommendations(indices[i],'parent_category_name',cosine_similarities,variant1,variant2)
df1['recommended_sku_v1'][i] = remove_same_category_products(recommeded_order_sku_v1,category_name)[:20]
df1['score_cs'][i] = score_cs
df1['order_score'][i] = order_score
# df1['recommended_sku_v2'][i] = remove_same_category_products(recommeded_order_sku_v2,category_name)[:20]


In [140]:
category_name

'Manish Malhotra Sandalwood SPF 25 Gel'

In [135]:
df1['order_score'][i] 

sku
8906095889006    0.147829
8906095887941    0.142392
Name: order_score, dtype: float64

In [136]:
df1['score_cs'][i] 

sku
8906095889006    0.320126
8906095887941    0.264562
Name: score_cs, dtype: float32

In [141]:
df1[df1['category_name']=='Manish Malhotra Sandalwood SPF 25 Gel']

,brand_name,parent_category_name,category_name,product_name,total_orders,shadelabel,coverage,concern,formulation,finish,...,keywords,title,description,subtitle,details,cleaned,base_price_norm,recommended_sku_v1,score_cs,order_score
sku,,,,,,,,,,,,,,,,,,,,,
8906095886630,Manish Malhotra,Sun Care,Manish Malhotra Sandalwood SPF 25 Gel,Manish Malhotra Sandalwood SPF 25 Gel,7108.0,None,None,None,None,None,...,"Sandalwood Sunscreen Gel, sunscreen gel-based,...",Buy Manish Malhotra Sandalwood Spf 25 Gel Onli...,Buy MyGlamm Manish Malhotra sandalwood SPF 25 ...,Gel-based Natural Sunscreen With SPF,Manish Malhotra Sandalwood SPF 25 Gel Sun Care...,manish malhotra sandalwood spf 25 gel sun care...,0.004601,"[8906095889006, 8906095887941]",sku 8906095889006 0.320126 8906095887941 ...,sku 8906095889006 0.147829 8906095887941 ...


In [88]:
df1[df1.index==indices[i]]['category_name'][indices[i]]

'POPxo Makeup Collection'

In [89]:
df1['recommended_sku_v1'][i]

array(['8906095884971', '8906095886692', '8906095884612', '8906095887606',
       '8906095883660', '8906095880607', '8906095881895', '8906095886708',
       '8906095886845', '8906095883639', '8906095889358', '8906095880621',
       '123456', '8906095885015', '8906095884605', '8906095880645',
       '8906095884728', '8906095881420', '8906095881147', '8906095881871'],
      dtype=object)

In [52]:
variant1 = 'order_score'
variant2 = 'base_price_score'

for i in range(0, len(indices)):
    category_name=df1[df1.index==indices[i]]['category_name'][indices[i]]
    recommeded_order_sku_v1 , recommeded_order_sku_v2 = recommendations(indices[i],'parent_category_name',cosine_similarities,variant1,variant2)
    df1['recommended_sku_v1'][i] = remove_same_category_products(recommeded_order_sku_v1,category_name)[:20]
    df1['recommended_sku_v2'][i] = remove_same_category_products(recommeded_order_sku_v2,category_name)[:20]

# df50=df1[['parent_category_name','category_name','product_name','base_price','recommended_sku']].reset_index(drop=True)


KeyboardInterrupt: 

In [144]:
df2=df1[['recommended_sku_v1','recommended_sku_v2']].reset_index()

In [145]:
# df2

In [51]:
df1

,brand_name,parent_category_name,category_name,product_name,total_orders,shadelabel,coverage,concern,formulation,finish,...,sourceid,keywords,title,description,subtitle,details,cleaned,base_price_norm,recommended_sku_v1,recommended_sku_v2
sku,,,,,,,,,,,,,,,,,,,,,
STPL1701001,MyGlamm,Makeup Brushes And Accessories,MyGlamm Manicure Kit,MyGlamm Manicure Kit,80.0,None,None,None,None,None,...,5e5e33b4f65d1b191c353118,"manicure kit products, manicure kit list, how ...",Manicure Tools Kit - 5 Piece Nail Manicure Ki...,This super cute Myglamm's manicure and pedicur...,Special Edition Manicure Set,MyGlamm Manicure Kit Makeup Brushes And Access...,myglamm manicure kit makeup brushes accessorie...,0.001750,[],[]
STPL1699001,MyGlamm,Makeup Brushes And Accessories,MyGlamm Vanity - Big,MyGlamm Vanity - Big,12.0,None,None,None,None,None,...,5e5e33b6bc3e53271a3a1597,,\tMyGlamm Vanity - Big,,,MyGlamm Vanity - Big Makeup Brushes And Acce...,myglamm vanity big makeup brushes accessories ...,0.000000,[],[]
8906095884223,Manish Malhotra,Lip Makeup | Eye Makeup | Face Makeup,Manish Malhotra Metallic Chalk Duo,Manish Malhotra Metallic Chalk Duo - Inferno,746.0,Inferno,Full,None,Powder,Metallic-matte,...,5e5f848b748b0d37129d15b7,"Manish Malhotra Metallic Chalk Duo, Manish Mal...",Buy Manish Malhotra Metallic Chalk Duo - Infer...,Buy MyGlamm Manish Malhotra Metallic Chalk Duo...,"Multifunctional Metallic Lip, Eye & Cheek Chalk",Manish Malhotra Metallic Chalk Duo - Inferno L...,manish malhotra metallic chalk duo inferno lip...,0.006750,[],[]
8906095886050,Treat Love Cared,Face Makeup,Treat Love Care Oil Control Foundation,Treat Love Care Oil Control Foundation - Renew,1987.0,Renew,None,None,None,None,...,5f756235e7e93c9c12f3b26e,"renew, foundation, renew oil control foundatio...",Buy Treat Love Care Oil Control Foundation - R...,Buy MyGlamm Treat Love Care Oil Control Founda...,"Liquid Foundation With Buildable Coverage, Enr...",Treat Love Care Oil Control Foundation - Renew...,treat love care oil control foundation renew f...,0.004975,[],[]
8906095886036,Treat Love Cared,Face Makeup,Treat Love Care 24 HR Anti-Pollution Foundation,Treat Love Care 24 HR Anti-Pollution Foundatio...,1025.0,Pure,None,None,None,None,...,5f756ef21cb5fb0a9a5cf747,"anti pollution foundation, myglamm anti-pollut...",Buy Treat Love Care 24 HR Anti-pollution Found...,Buy MyGlamm Treat Love Care 24 HR Anti-polluti...,"Liquid Foundation With Buildable Coverage, Enr...",Treat Love Care 24 HR Anti-Pollution Foundatio...,treat love care 24 hr anti pollution foundatio...,0.004975,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8906095889440,Manish Malhotra,Nail Polish Sets,Manish Malhotra Gel Finish Nail Lacquer - The ...,Manish Malhotra Gel Finish Nail Lacquer - The ...,233.0,Red Mystique,None,None,None,None,...,61fa9604eb827bf262790a5f,,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,Buy Manish Malhotra gel finish nail lacquer - ...,"Ultra-Glossy, Long-Lasting Gel Nail Enamel",Manish Malhotra Gel Finish Nail Lacquer - The ...,manish malhotra gel finish nail lacquer love e...,0.001475,[],[]
8906095889464,Manish Malhotra,Nail Polish Sets,Manish Malhotra Gel Finish Nail Lacquer - The ...,Manish Malhotra Gel Finish Nail Lacquer - The ...,967.0,Sienna Crush,None,None,None,None,...,61fb570fde45242639a8629f,,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,Buy Manish Malhotra gel finish nail lacquer - ...,"Ultra-Glossy, Long-Lasting Gel Nail Enamel",Manish Malhotra Gel Finish Nail Lacquer - The ...,manish malhotra gel finish nail lacquer love e...,0.001475,[],[]
8906095889501,Manish Malhotra,Nail Polish Sets,Manish Malhotra Gel Finish Nail Lacquer - The ...,Manish Malhotra Gel Finish Nail Lacquer - The ...,759.0,Persian Rosé,None,None,None,None,...,61fb58a3de45242639a862ab,,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,Buy Manish Malhotra Gel Finish Nail Lacquer - ...,"Ultra-Glossy, Long-Lasting Gel Nail Enamel",Manish Malhotra Gel Finish

In [43]:
df1.columns

Index(['brand_name', 'parent_category_name', 'category_name', 'product_name',
       'total_orders', 'shadelabel', 'coverage', 'concern', 'formulation',
       'finish', 'skintype', 'ingredient', 'color', 'base_price',
       'vendor_code', 'sourceid', 'keywords', 'title', 'description',
       'subtitle', 'details', 'cleaned', 'base_price_norm',
       'recommended_sku_v1', 'recommended_sku_v2'],
      dtype='object')

In [146]:
# prepare products details dictionary with sku as key and details as its value as another dictionary

product_details_query = '''
select distinct t1.*, t2.totalCount, t2.avgRating from (
select 
    p.sku,
    p.brand_name,
    p.parent_category_name,
    p.category_name,
    p.product_name,
    p.product_url,
    p.base_price as priceMRP,
    p.offer_price as priceOffer,
    p.product_url as url,
    p.shadelabel as shade,
    c.productTitle,
    c.productSubTitle,
    c.productName,
    s.slug,
    imageURL,
    videoURL,
    pp.category_rank
from 
    `myglamm-india.master_tables.master_product_table_v2` p
left outer join (
    select 
        sourceid as product_id,
        data.metadata.title as productTitle,
        data.content.subtitle as productSubTitle,
        data.content.name as productName
    from 
        `myglamm-india.production_mongo_v2.Content`, unnest(properties.filter) as pf
    where   
        source = 'product' 
        and pf.value.language = 'EN'
) c on p.product_id = c.product_id
left outer join (
    select 
        sourceid as product_id,
        url as slug
    from    
        `myglamm-india.production_mongo_v2.Url`
    where 
        source = 'product' 
) s on p.product_id = s.product_id
left outer join (
    select 
        product_id, 
        imageURL 
    from (
        select 
            _id,
            sourceid as product_id,
            imageurl._400x400 as imageURL,
            displayorder,
            RANK() OVER ( PARTITION BY sourceid ORDER BY displayorder ) AS rank
        from 
            `myglamm-india.production_mongo_v2.asset`
        where 
            source = 'product' 
            and statusid = 1 
            and type = 'image'
        order by 
            displayOrder asc
    ) 
    where 
        rank = 1
) i on p.product_id = i.product_id
left outer join (
    select 
        product_id, 
        videoURL 
    from (
        select 
            _id,
            sourceid as product_id,
            url as videoURL,
            displayorder,
            RANK() OVER ( PARTITION BY sourceid ORDER BY displayorder ) AS rank
        from 
            `myglamm-india.production_mongo_v2.asset`
        where 
            source = 'product' 
            and statusid = 1 
            and type = 'video'
        order by 
            displayOrder asc
    ) where rank = 1
) v on p.product_id = v.product_id
left outer join 
    `myglamm-india.mg_reports_primary.product_persona` pp on p.sku = pp.sku
where 
    p.product_status = 'Active' and p.vendor_code='mgp'
) t1   
left join (
select * from (
SELECT distinct
        case
            when meta.sku is null and meta.sku__st is null and reviewerinfo.additional.sku is not null then cast(reviewerinfo.additional.sku as string)
            when meta.sku is null then meta.sku__st
            else cast(meta.sku as string)
        end as sku,
        meta.productname,
        count(*) as totalCount,
        round(avg(rating),2) as avgRating
    FROM
        `myglamm-india.authorized_views.reviews_av`
    group by
        1, 2
) where sku is not null
) t2 on t1.sku=t2.sku and t1.product_name=t2.productname
where t1.sku not in (
    select distinct sku from (
        SELECT 
          sku 
        FROM `myglamm-india.production_mongo_v2.Products` 
        where productmeta.isessential=true

        union all

        SELECT  
          sku
        FROM `myglamm-india.production_mongo_v2.comboProduct` 
        where productmeta.isessential=true

        union all
         
        select sku from `myglamm-india.master_tables.master_product_table_v2`  where sku not in (SELECT sku FROM `myglamm-india.master_tables.master_product_table_v2` 
        where hidden=true 
        and in_stock=true
        )

    )
)
order by 
    t1.category_rank
    
'''

product_details_job = client.query(product_details_query)
product_details_json = product_details_job.result()
product_details_json_dict = {}

In [147]:
for row in product_details_json:
    # print(row)
    product_details_json_dict[row['sku']] = {
        'sku': row['sku'],
        'imageURL': row['imageURL'],
        'videoURL': row['videoURL'],
        'productName': row['productName'],
        'productTitle': row['productTitle'],
        'productSubTitle': row['productSubTitle'],
        'slug': row['slug'],
        'priceMRP': row['priceMRP'],
        'priceOffer': row['priceOffer'],
        'url': row['url'],
        'shade': row['shade'],
        'totalCount': row['totalCount'],
        'avgRating': row['avgRating']
    }


In [148]:
# df2

In [149]:
similar_products_v1 = {}

for x in df2.iterrows():
    sku = x[1][0]
    recommended_sku_v1 = x[1][1]

#     print('sku')
#     print(sku)
#     print('recommended_sku')
#     print(recommended_sku_v1)

    similar_products_v1[sku] = []
    for rsku in recommended_sku_v1:
#         print('sku')
#         print(sku)
#         print('rsku')
#         print(rsku)
        if rsku in product_details_json_dict:
            product_details_json_dict[rsku]
            smprod = {
                "sku":product_details_json_dict[rsku]['sku'],
                "imageURL":product_details_json_dict[rsku]['imageURL'],
                "videoURL":product_details_json_dict[rsku]['videoURL'],
                "productName":product_details_json_dict[rsku]['productName'],
                "productTitle":product_details_json_dict[rsku]['productTitle'],
                "productSubTitle":product_details_json_dict[rsku]['productSubTitle'],
                "slug":product_details_json_dict[rsku]['slug'],
                "priceMRP":product_details_json_dict[rsku]['priceMRP'],
                "priceOffer":product_details_json_dict[rsku]['priceOffer'],
                "url":product_details_json_dict[rsku]['url'],
                "shade":product_details_json_dict[rsku]['shade']
            }

            if 'totalCount' in  product_details_json_dict[rsku] and 'avgRating' in product_details_json_dict[rsku] and product_details_json_dict[rsku]['totalCount'] != None and product_details_json_dict[rsku]['avgRating'] != None:
                smprod['rating'] = {
                    "totalCount": product_details_json_dict[rsku]['totalCount'],
                    "avgRating": product_details_json_dict[rsku]['avgRating']
                }
            
            similar_products_v1[sku].append(smprod)


# print(similar_products)

f = open("similar_products_v1.txt", "a")
f.write(json.dumps(similar_products_v1))
f.close()

# # df1.to_csv('similar_products.csv',index=False,sep="\t")
# print(df1)

In [150]:
similar_products_v2 = {}

for x in df2.iterrows():
    sku = x[1][0]
    recommended_sku_v2 = x[1][2]

#     print('sku')
#     print(sku)
#     print('recommended_sku')
#     print(recommended_sku_v1)

    similar_products_v2[sku] = []
    for rsku in recommended_sku_v2:
#         print('sku')
#         print(sku)
#         print('rsku')
#         print(rsku)
        if rsku in product_details_json_dict:
            product_details_json_dict[rsku]
            smprod = {
                "sku":product_details_json_dict[rsku]['sku'],
                "imageURL":product_details_json_dict[rsku]['imageURL'],
                "videoURL":product_details_json_dict[rsku]['videoURL'],
                "productName":product_details_json_dict[rsku]['productName'],
                "productTitle":product_details_json_dict[rsku]['productTitle'],
                "productSubTitle":product_details_json_dict[rsku]['productSubTitle'],
                "slug":product_details_json_dict[rsku]['slug'],
                "priceMRP":product_details_json_dict[rsku]['priceMRP'],
                "priceOffer":product_details_json_dict[rsku]['priceOffer'],
                "url":product_details_json_dict[rsku]['url'],
                "shade":product_details_json_dict[rsku]['shade']
            }

            if 'totalCount' in  product_details_json_dict[rsku] and 'avgRating' in product_details_json_dict[rsku] and product_details_json_dict[rsku]['totalCount'] != None and product_details_json_dict[rsku]['avgRating'] != None:
                smprod['rating'] = {
                    "totalCount": product_details_json_dict[rsku]['totalCount'],
                    "avgRating": product_details_json_dict[rsku]['avgRating']
                }
            
            similar_products_v2[sku].append(smprod)


# print(similar_products)

f = open("similar_products_v2.txt", "a")
f.write(json.dumps(similar_products_v2))
f.close()

# # df1.to_csv('similar_products.csv',index=False,sep="\t")
# print(df1)

In [151]:
# similar_products_v2

In [152]:
dump_requests = []

for key in similar_products_v1:
    
    if len(similar_products_v1[key]):
        curl_req_body = {
            "identifier": key,
            "key": "similarProducts",
            "vendorCode": "mgp",
            "value": {
                "variantValue": "similarproductsv1",
                "products": similar_products_v1[key]
            },
        }
        dump_requests.append(curl_req_body)
        # print(curl_req_body)

In [153]:
len(dump_requests)

560

In [159]:
dump_requests[10]

{'identifier': '8906095886937',
 'key': 'similarProducts',
 'vendorCode': 'mgp',
 'value': {'variantValue': 'similarproductsv1',
  'products': [{'sku': '8906095881741',
    'imageURL': 'https://files.myglamm.com/site-images/400x400/Hi-Shine-Post-1-Caramel-kiss_1.jpg',
    'videoURL': 'youtu.be/SCYULyGXdkA',
    'productName': 'Manish Malhotra Hi-Shine Lipstick - Caramel Kiss',
    'productTitle': 'Buy Manish Malhotra Hi-Shine Lipstick - Caramel Kiss  (Caramel Tan) Online at Best Price - MyGlamm',
    'productSubTitle': 'Lightweight, Long-wear Lipsticks',
    'slug': '/product/manish-malhotra-hi-shine-lipstick-caramel-kiss.html',
    'priceMRP': 950.0,
    'priceOffer': 665.0,
    'url': 'https://myglamm.in/MpNsLoIWw4',
    'shade': 'Caramel Kiss',
    'rating': {'totalCount': 5, 'avgRating': 4.0}},
   {'sku': '8906095882229',
    'imageURL': 'https://files.myglamm.com/site-images/400x400/Sinner-(1)_1.jpg',
    'videoURL': 'https://www.youtube.com/watch?v=Q4gVQWfiiWc',
    'productName'

In [157]:
import requests
import json
# for i in dump_requests:

headers = {
    'Content-Type': 'application/json',
    'apiKey': 'fac2c144a7da65d81c939597bf983613',
} 


# response = requests.post('https://mag.myglamm.net/dump-ms/dump', headers=headers, data=payload)

In [158]:
for i in range(0,len(dump_requests)):
    df_temp=dump_requests[i]
    payload = json.dumps([df_temp])
    print("..............................",i,"..........................")
    response = requests.post('https://acl.mgapis.com/dump-ms/dump', headers=headers, data=payload)
    print(response.text)

.............................. 0 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 1 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 2 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 3 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 4 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 5 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"statu

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 48 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 49 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 50 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 51 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 52 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.................

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 96 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 97 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 98 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 99 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 100 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
................

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 143 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 144 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 145 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 146 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 147 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
............

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 190 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 191 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 192 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 193 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 194 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
............

...

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 331 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 332 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 333 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 334 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 335 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
............

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 378 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 379 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 380 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 381 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 382 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
............

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 425 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 426 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 427 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 428 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 429 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
............

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 472 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 473 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 474 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 475 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 476 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
............

{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 519 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 520 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 521 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 522 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
.............................. 523 ..........................
{"name":"OK","status":true,"code":200,"message":"Processing","data":{"status":"SUCCESS","message":"Processing"}}
............